<a href="https://colab.research.google.com/github/N2026-2025/Homework4-Zoomcamp/blob/main/homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#BIG QUERY/SANDBOX
import json
import pandas as pd
import os
from google.colab import userdata
from google.cloud import bigquery
from google.oauth2 import service_account

# Cargar el JSON desde el Secret de Colab
secret_content = userdata.get('GCP_CREDENTIALS')
info = json.loads(secret_content)
credentials = service_account.Credentials.from_service_account_info(info)

# Crear el cliente de BigQuery
client = bigquery.Client(credentials=credentials, project=info['project_id'])

# CONFIGURACIÓN: Cambiá esto si tu dataset tiene otro nombre
DATASET_ID = "nyctaxi"


In [5]:
# Alternative Listar todos los archivos .parquet que subiste a Colab
archivos_locales = [f for f in os.listdir('.') if f.endswith('.parquet')]

if not archivos_locales:
    print("❌ No encontré archivos .parquet. ¡Subilos a la carpeta de la izquierda!")
else:
    for archivo in archivos_locales:
        # Definir el nombre de la tabla según el archivo
        # Ejemplo: yellow_tripdata_2019-01.parquet -> tabla: yellow_tripdata
        table_name = "yellow_tripdata" if "yellow" in archivo.lower() else "green_tripdata"
        table_ref = f"{client.project}.{DATASET_ID}.{table_name}"

        print(f"⏳ Subiendo {archivo} a {table_ref}...")

        # Leer el archivo local
        df = pd.read_parquet(archivo)

        # Configurar la carga
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND", # Agrega datos si la tabla ya existe
            autodetect=True                  # Crea las columnas automáticamente
        )

        # Ejecutar la carga directa
        try:
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result() # Esperar a que termine
            print(f"✅ ¡Éxito! Se cargaron {len(df)} filas desde {archivo}.")
        except Exception as e:
            print(f"❌ Error con {archivo}: {e}")

print("\n--- Proceso finalizado ---")


❌ No encontré archivos .parquet. ¡Subilos a la carpeta de la izquierda!

--- Proceso finalizado ---


In [16]:
import pandas as pd
import requests
from io import BytesIO
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import userdata
import json

# --- 1. CONFIGURACIÓN DE INFRAESTRUCTURA ---
GITHUB_USER = "DataTalksClub"
REPO_NAME = "nyc-tlc-data"
# Agregamos la / después de .com para que la URL sea válida
BASE_URL = f"https://github.com/{GITHUB_USER}/{REPO_NAME}/releases/download"

# --- 2. CREDENCIALES Y CLIENTE BQ ---
info = json.loads(userdata.get('GCP_CREDENTIALS'))
credentials = service_account.Credentials.from_service_account_info(info)
client = bigquery.Client(credentials=credentials, project=info['project_id'])
DATASET_ID = "nyctaxi"

# --- 3. ESQUEMA DE DATOS (Dtypes de tu ejemplo) ---
dtype_dict = {
    "VendorID": "Int64", "passenger_count": "Int64", "trip_distance": "float64",
    "RatecodeID": "Int64", "store_and_fwd_flag": "string", "PULocationID": "Int64",
    "DOLocationID": "Int64", "payment_type": "Int64", "fare_amount": "float64",
    "extra": "float64", "mta_tax": "float64", "tip_amount": "float64",
    "tolls_amount": "float64", "improvement_surcharge": "float64",
    "total_amount": "float64", "congestion_surcharge": "float64"
}

def ingest_taxi_data():
    for color in ['yellow', 'green']:
      for year in [2019, 2020]: # 2019 y 2020
            for month in range(1, 13):
                file_name = f"{color}_tripdata_{year}-{month:02d}.csv.gz"
                url = f"{BASE_URL}/{color}/{file_name}"

                table_ref = f"{client.project}.{DATASET_ID}.{color}_tripdata"
                date_col = 'tpep_pickup_datetime' if color == 'yellow' else 'lpep_pickup_datetime'

                # --- CONTROL DE DUPLICADOS ---
                try:
                    query = f"SELECT count(1) as total FROM `{table_ref}` WHERE EXTRACT(YEAR FROM {date_col}) = {year} AND EXTRACT(MONTH FROM {date_col}) = {month}"
                    if list(client.query(query).result()).total > 0:
                        print(f"⏭️ Saltando {file_name}: Ya existe en BigQuery.")
                        continue
                except Exception: pass

                # --- DESCARGA E INGESTA ---
                print(f"⏳ Intentando descargar desde: {url}")
                try:
                    response = requests.get(url, timeout=60)
                    if response.status_code == 200:
                        # Leemos el CSV usando tus dtypes y parse_dates
                        df = pd.read_csv(
                            BytesIO(response.content),
                            compression='gzip',
                            dtype=dtype_dict,
                            low_memory=False
                        )

                        # Convertimos las fechas (dinámico según color)
                        df[date_col] = pd.to_datetime(df[date_col])
                        df[date_col.replace('pickup', 'dropoff')] = pd.to_datetime(df[date_col.replace('pickup', 'dropoff')])

                        # Carga a BigQuery
                        job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND", autodetect=True)
                        client.load_table_from_dataframe(df, table_ref, job_config=job_config).result()
                        print(f"✅ Descarga e Ingesta exitosa de {file_name}")
                    else:
                        print(f"❌ Error {response.status_code} en la URL. Verificá que el archivo exista.")
                except Exception as e:
                    print(f"⚠️ Error técnico en {file_name}: {e}")

# EJECUTAR
ingest_taxi_data()


⏳ Intentando descargar desde: https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-01.csv.gz
✅ Descarga e Ingesta exitosa de yellow_tripdata_2019-01.csv.gz
⏳ Intentando descargar desde: https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-02.csv.gz
✅ Descarga e Ingesta exitosa de yellow_tripdata_2019-02.csv.gz
⏳ Intentando descargar desde: https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-03.csv.gz
✅ Descarga e Ingesta exitosa de yellow_tripdata_2019-03.csv.gz
⏳ Intentando descargar desde: https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-04.csv.gz
✅ Descarga e Ingesta exitosa de yellow_tripdata_2019-04.csv.gz
⏳ Intentando descargar desde: https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-05.csv.gz
✅ Descarga e Ingesta exitosa de yellow_tripdata_2019-05.csv.gz
⏳ Intentando descargar de